In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated
from langgraph.types import Command
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [ ]:
class State(TypedDict):
    input: str
    user_feedback : str

In [ ]:
llm = ChatGroq(model="llama-3.1-8b-instant")
llm

In [ ]:
def step_1(state:State):
    print('---step1----')
    pass

def human_feedback(state:State):
    '''Request assistance from human.'''
    print('---Human Feedback----')
    pass

def step_3(state:State):
    print("---step3---")
    pass

In [ ]:
builder = StateGraph(State)
builder.add_node("1",step_1)
builder.add_node("human",human_feedback)
builder.add_node("3",step_3)
builder.add_edge(START,"1")
builder.add_edge("1","human")
builder.add_edge("human","3")
builder.add_edge("3",END)

memory = MemorySaver()


graph = builder.compile(checkpointer=memory,interrupt_before=['human'])


In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
user_input = "I need some expert guidance and assistance for building an AI agent. Could you request assistance for me?"
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": user_input},
    config=config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

Here the output is just step1 meaning the first node executes but it requires human assistance and hence interrupted before step2 could execute.

Now we will provide the required human assistance (in the form of command for now)

In [ ]:

human_response = (
    "We, the experts are here to help! We'd recommend you check out LangGraph to build your agent."
    " It's much more reliable and extensible than simple autonomous agents."
)

human_command = Command(resume={"data": human_response})

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

Hence after providing the Human command for the assistance, the step 3 got executed as well.